In [17]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from keras.models import Sequential
from keras.initializers import Initializer
from sklearn.cluster import KMeans
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
import math
from keras.datasets import boston_housing
from keras import layers
from keras import backend as K
import keras_tuner as kt
from keras.layers import Dense, Dropout


class RBFLayer(layers.Layer):

    def __init__(self, output_dim, initializer=None, betas=1.0, **kwargs):
        self.output_dim = output_dim
        self.init_betas = betas
        if not initializer:
            self.initializer = RandomUniform(0.0, 1.0)
        else:
            self.initializer = initializer
        super(RBFLayer, self).__init__(**kwargs)

    def build(self, input_shape):

        self.centers = self.add_weight(name='centers',
                                       shape=(self.output_dim, input_shape[1]),
                                       initializer=self.initializer,
                                       trainable=True)
        d_max = 0
        for i in range(0, self.output_dim):
            for j in range(0, self.output_dim):
                d = np.linalg.norm(self.centers[i] - self.centers[j])
                if d > d_max:
                    d_max = d
        sigma = d_max / np.sqrt(2 * self.output_dim)
        self.betas = np.ones(self.output_dim) / (2 * (sigma ** 2))
        
        super(RBFLayer, self).build(input_shape)

    def call(self, x):

        C = K.expand_dims(self.centers)
        H = K.transpose(C-K.transpose(x))
        return K.exp(-self.betas * K.sum(H**2, axis=1))

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)



def rmse(y_true, y_pred):
	return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1))
 

class Init_Centers_KMeans(Initializer):
    """ Initializer for initialization of centers of RBF network
        by clustering the given data set.
    # Arguments
        X: dataset
    """

    def __init__(self, X, max_iter=100):
        self.X = X
        self.max_iter = max_iter

    def __call__(self, shape, dtype=None):
        assert shape[1] == self.X.shape[1]

        n_centers = shape[0]
        km = KMeans(n_clusters=n_centers, max_iter=self.max_iter, verbose=0)
        km.fit(self.X)
        print(n_centers)
        return km.cluster_centers_



(x_train, y_train), (x_test, y_test) = boston_housing.load_data(test_split=0.25)

#z-score normalization
x_train = (x_train - np.mean(x_train, axis=0)) / np.std(x_train, axis=0)
x_test = (x_test - np.mean(x_test, axis=0)) / np.std(x_test, axis=0)


def model_builder(hp):

  model = Sequential()

  # Tune the number of units in the RBF layer
  # Choose an optimal value from 19, 57, 114, 190
  hp_units_1 = hp.Choice('units_1', values=[19, 57, 114, 190])

  # Tune the number of units in the second Dense layer
  # Choose an optimal value from 32, 64, 128, 256
  hp_units_2 = hp.Choice('units_2', values=[32, 64, 128, 256])

  #Tune parameter rate for Dropout
  #Choose an optimal value from 0.2, 0.35, 0.5
  hp_rate = hp.Choice('rate', values=[0.2, 0.35, 0.5])

  rbflayer = RBFLayer(output_dim=hp_units_1, initializer=Init_Centers_KMeans(x_train), input_shape=(13,))
  model.add(rbflayer)
  model.add(Dense(units=hp_units_2))
  model.add(Dropout(rate=hp_rate))
  model.add(Dense(1))


  model.compile(loss='mean_squared_error', optimizer = keras.optimizers.SGD(0.001), metrics=[rmse])

  return model

tuner = kt.Hyperband(model_builder, objective=kt.Objective("rmse", direction="min"), max_epochs=2)
  
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=200)

tuner.search(x_train, y_train, epochs=100, validation_split=0.2, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units 
in the RBF
layer is {best_hps.get('units_1')}, the optimal number of units 
in the second densely-connected
layer is {best_hps.get('units_2')} and the optimal rate parameter for dropout is {best_hps.get('rate')}.
""")

#Build the model with the optimal hyperparameters and train it on the data for 100 epochs
model = tuner.hypermodel.build(best_hps)

history = model.fit(x_train, y_train, batch_size=4, epochs=100, validation_split=0.2)

# summarize history for accuracy
plt.plot(history.history['rmse'])
plt.plot(history.history['val_rmse'])
plt.title('model rmse')
plt.ylabel('rmse')
plt.xlabel('epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

INFO:tensorflow:Reloading Oracle from existing project ./untitled_project/oracle.json
19
INFO:tensorflow:Reloading Tuner from ./untitled_project/tuner0.json
INFO:tensorflow:Oracle triggered exit

The hyperparameter search is complete. The optimal number of units 
in the RBF
layer is 114, the optimal number of units 
in the second densely-connected
layer is 32 and the optimal rate parameter for dropout is 0.2.

114


KeyboardInterrupt: ignored